<a href="https://colab.research.google.com/github/Janardhan97/ATM_FRAUDLENT_TRANSCTIONS/blob/main/ATM_FRAUDLENT_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###Problem Statement: PredCatch Analytics' Australian banking client's profitability and reputation are being hit by fraudulent ATM transactions. 
##They want PredCatch to help them in reducing and if possible completely eliminating such fraudulent transactions.
## PredCatch believes it can do the same by building a predictive model to catch such fraudulent transactions in real time and decline them.
 ##Your job as PredCatch's Data Scientist is to build this fraud detection & prevention predictive model 
 
##The data has been made available to you. 
##The challenging part of the problem is that the data contains very few fraud instances in comparison to the overall population. To give more edge to the solution they have also collected data regarding location [geo_scores] of the transactions, their own proprietary index [Lambda_wts], on network turn around times [Qset_tats] and vulnerability qualification score [instance_scores]. As of now you don't need to understand what they mean.
##Training data contains masked variables pertaining to each transaction id . Your prediction target here is 'Target' .

##1: Fraudulent transactions


####0: Clean transactions


In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Projects Data Science/ATM Fraudlent/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Projects Data Science/ATM Fraudlent/test_share.csv')
geo = pd.read_csv('/content/drive/MyDrive/Projects Data Science/ATM Fraudlent/Geo_scores.csv')
lambdawts = pd.read_csv('/content/drive/MyDrive/Projects Data Science/ATM Fraudlent/Lambda_wts.csv')
qsets = pd.read_csv('/content/drive/MyDrive/Projects Data Science/ATM Fraudlent/Qset_tats.csv')
instance = pd.read_csv('/content/drive/MyDrive/Projects Data Science/ATM Fraudlent/instance_scores.csv')


In [4]:
train

#CHECK THE DATA AND FIND THE PATTERNS

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem8,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,Target
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.680000,0.726667,0.606667,1.010000,0.933333,0.603333,0.686667,0.673333,-245.7500,0
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.716667,0.743333,0.680000,0.690000,0.560000,0.670000,0.553333,0.653333,-248.0000,0
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.690000,0.820000,0.600000,0.383333,0.763333,0.670000,0.686667,0.673333,-233.1250,0
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.720000,0.900000,0.680000,0.846667,0.423333,0.520000,0.846667,0.760000,-249.7775,0
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.603333,0.486667,0.693333,0.526667,0.520000,0.716667,0.706667,0.673333,-247.5775,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227840,97346,Grp232,0.476667,1.013333,0.536667,0.576667,1.406667,1.846667,0.600000,1.103333,...,0.683333,0.630000,0.633333,0.996667,0.646667,0.533333,0.680000,0.693333,-246.5025,0
227841,147361,Grp199,1.363333,0.730000,0.060000,0.776667,0.883333,0.466667,0.733333,0.590000,...,0.543333,0.356667,0.766667,0.730000,0.596667,0.730000,0.646667,0.656667,-249.7775,0
227842,50989,Grp36,1.060000,0.756667,0.906667,0.896667,0.503333,0.396667,0.683333,0.620000,...,0.603333,0.510000,0.740000,0.873333,0.700000,0.696667,0.663333,0.673333,-249.7775,0
227843,149780,Grp445,0.433333,1.013333,1.163333,0.940000,0.930000,0.900000,0.813333,0.720000,...,0.606667,0.606667,0.540000,0.643333,0.906667,0.540000,0.766667,0.710000,-242.7500,0


In [5]:
test

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem7,Dem8,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT
0,146574,Grp229,-0.300000,1.540000,0.220000,-0.280000,0.570000,0.260000,0.700000,1.076667,...,0.786667,0.546667,0.313333,0.703333,0.813333,0.776667,0.796667,0.823333,0.783333,-249.7500
1,268759,Grp141,0.633333,0.953333,0.810000,0.466667,0.910000,0.253333,1.040000,0.550000,...,0.636667,0.770000,0.993333,0.536667,0.703333,0.806667,0.630000,0.673333,0.673333,-249.8125
2,59727,Grp188,1.043333,0.740000,0.860000,1.006667,0.583333,0.616667,0.630000,0.686667,...,0.626667,0.756667,0.953333,0.623333,0.753333,0.870000,0.596667,0.680000,0.670000,-248.1200
3,151544,Grp426,1.283333,0.300000,0.576667,0.636667,0.256667,0.543333,0.356667,0.663333,...,0.480000,0.460000,0.260000,0.800000,0.606667,0.456667,0.320000,0.676667,0.660000,-222.9875
4,155008,Grp443,1.186667,0.326667,0.476667,0.866667,0.436667,0.680000,0.476667,0.686667,...,0.706667,0.740000,0.823333,0.670000,0.896667,0.566667,0.546667,0.650000,0.663333,-196.2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56957,18333,Grp102,0.553333,1.043333,1.096667,0.686667,0.673333,0.340000,0.900000,0.643333,...,0.700000,0.576667,0.433333,0.660000,0.776667,0.610000,0.690000,0.750000,0.700000,-249.5050
56958,244207,Grp504,1.353333,0.616667,0.276667,0.783333,0.690000,0.650000,0.473333,0.670000,...,0.643333,0.713333,0.870000,0.683333,0.690000,0.640000,0.883333,0.663333,0.660000,-248.7525
56959,103277,Grp78,1.083333,0.433333,0.806667,0.490000,0.243333,0.316667,0.533333,0.606667,...,0.536667,0.433333,0.063333,0.753333,0.780000,0.603333,0.880000,0.643333,0.676667,-231.0500
56960,273294,Grp134,0.566667,1.153333,0.370000,0.616667,0.793333,0.226667,0.910000,0.696667,...,0.643333,0.776667,1.026667,0.626667,0.646667,0.566667,0.616667,0.713333,0.706667,-246.3150


In [6]:
geo

,id,geo_score
0,26674,4.48
1,204314,4.48
2,176521,5.17
3,48812,-2.41
4,126870,6.55
...,...,...
1424030,107880,1.03
1424031,282410,8.62
1424032,209634,-1.72
1424033,211652,-10.00


In [7]:
lambdawts

,Group,lambda_wt
0,Grp936,3.41
1,Grp347,-2.88
2,Grp188,0.39
3,Grp1053,-2.75
4,Grp56,-0.83
...,...,...
1395,Grp892,4.24
1396,Grp1072,-7.28
1397,Grp785,-2.63
1398,Grp50,0.79


In [8]:
qsets

,id,qsets_normalized_tat
0,9983,2.41
1,266000,3.10
2,77525,1.03
3,160765,-11.63
4,138220,-4.48
...,...,...
1424030,13009,NaN
1424031,147556,-7.24
1424032,143331,10.00
1424033,248231,-7.93


In [9]:
instance

,id,instance_scores
0,173444,-0.88
1,259378,1.50
2,161170,0.44
3,191161,0.76
4,34521,-0.84
...,...,...
1424030,80337,0.20
1424031,216371,0.64
1424032,35358,0.77
1424033,159766,10.56


In [10]:
geo['id'].value_counts().head()

26674     5
149679    5
114110    5
24969     5
262179    5
Name: id, dtype: int64

In [11]:
instance['id'].value_counts().head()

173444    5
240342    5
147158    5
83677     5
127535    5
Name: id, dtype: int64

In [12]:
qsets['id'].value_counts().head()

9983      5
146106    5
133237    5
262326    5
79108     5
Name: id, dtype: int64

In [13]:
lambdawts['Group'].value_counts().head()

Grp936     1
Grp1128    1
Grp341     1
Grp63      1
Grp173     1
Name: Group, dtype: int64

In [14]:
geo['geo_score'].isnull().sum()

# CHECK FULL NULL VALUES IN DATA

71543

In [15]:
instance['instance_scores'].isnull().sum()

0

In [16]:
lambdawts['lambda_wt'].isnull().sum()

0

In [17]:
qsets['qsets_normalized_tat'].isnull().sum()

103201

In [18]:
#filling  the missing value by using mean/medain
# for simplicity, I am filling with zero

In [19]:
geo.fillna(0, inplace=True)

qsets.fillna(0, inplace=True)

geo = geo.groupby('id').mean()

geo.head()

,geo_score
id,
0,-0.62
1,1.07
2,0.07
3,0.18
4,0.54


In [20]:
qsets = qsets.groupby('id').mean()
qsets.head()

,qsets_normalized_tat
id,
0,0.21
1,-0.11
2,1.11
3,-0.68
4,-0.24


In [21]:
instance = instance.groupby('id').mean()
train.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem8,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,Target
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.680000,0.726667,0.606667,1.010000,0.933333,0.603333,0.686667,0.673333,-245.7500,0
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.716667,0.743333,0.680000,0.690000,0.560000,0.670000,0.553333,0.653333,-248.0000,0
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.690000,0.820000,0.600000,0.383333,0.763333,0.670000,0.686667,0.673333,-233.1250,0
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.720000,0.900000,0.680000,0.846667,0.423333,0.520000,0.846667,0.760000,-249.7775,0
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.603333,0.486667,0.693333,0.526667,0.520000,0.716667,0.706667,0.673333,-247.5775,0


In [22]:
train['data'] = 'train'
train.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,Target,data
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.726667,0.606667,1.010000,0.933333,0.603333,0.686667,0.673333,-245.7500,0,train
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.743333,0.680000,0.690000,0.560000,0.670000,0.553333,0.653333,-248.0000,0,train
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.820000,0.600000,0.383333,0.763333,0.670000,0.686667,0.673333,-233.1250,0,train
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.900000,0.680000,0.846667,0.423333,0.520000,0.846667,0.760000,-249.7775,0,train
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.486667,0.693333,0.526667,0.520000,0.716667,0.706667,0.673333,-247.5775,0,train


In [23]:
test.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem7,Dem8,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT
0,146574,Grp229,-0.300000,1.540000,0.220000,-0.280000,0.570000,0.260000,0.700000,1.076667,...,0.786667,0.546667,0.313333,0.703333,0.813333,0.776667,0.796667,0.823333,0.783333,-249.7500
1,268759,Grp141,0.633333,0.953333,0.810000,0.466667,0.910000,0.253333,1.040000,0.550000,...,0.636667,0.770000,0.993333,0.536667,0.703333,0.806667,0.630000,0.673333,0.673333,-249.8125
2,59727,Grp188,1.043333,0.740000,0.860000,1.006667,0.583333,0.616667,0.630000,0.686667,...,0.626667,0.756667,0.953333,0.623333,0.753333,0.870000,0.596667,0.680000,0.670000,-248.1200
3,151544,Grp426,1.283333,0.300000,0.576667,0.636667,0.256667,0.543333,0.356667,0.663333,...,0.480000,0.460000,0.260000,0.800000,0.606667,0.456667,0.320000,0.676667,0.660000,-222.9875
4,155008,Grp443,1.186667,0.326667,0.476667,0.866667,0.436667,0.680000,0.476667,0.686667,...,0.706667,0.740000,0.823333,0.670000,0.896667,0.566667,0.546667,0.650000,0.663333,-196.2200


In [24]:
test['data'] = 'test'

test.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem8,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,data
0,146574,Grp229,-0.300000,1.540000,0.220000,-0.280000,0.570000,0.260000,0.700000,1.076667,...,0.546667,0.313333,0.703333,0.813333,0.776667,0.796667,0.823333,0.783333,-249.7500,test
1,268759,Grp141,0.633333,0.953333,0.810000,0.466667,0.910000,0.253333,1.040000,0.550000,...,0.770000,0.993333,0.536667,0.703333,0.806667,0.630000,0.673333,0.673333,-249.8125,test
2,59727,Grp188,1.043333,0.740000,0.860000,1.006667,0.583333,0.616667,0.630000,0.686667,...,0.756667,0.953333,0.623333,0.753333,0.870000,0.596667,0.680000,0.670000,-248.1200,test
3,151544,Grp426,1.283333,0.300000,0.576667,0.636667,0.256667,0.543333,0.356667,0.663333,...,0.460000,0.260000,0.800000,0.606667,0.456667,0.320000,0.676667,0.660000,-222.9875,test
4,155008,Grp443,1.186667,0.326667,0.476667,0.866667,0.436667,0.680000,0.476667,0.686667,...,0.740000,0.823333,0.670000,0.896667,0.566667,0.546667,0.650000,0.663333,-196.2200,test


In [25]:
all_data = pd.concat([train, test], axis=0)
all_data.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,Target,data
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.726667,0.606667,1.010000,0.933333,0.603333,0.686667,0.673333,-245.7500,0.0,train
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.743333,0.680000,0.690000,0.560000,0.670000,0.553333,0.653333,-248.0000,0.0,train
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.820000,0.600000,0.383333,0.763333,0.670000,0.686667,0.673333,-233.1250,0.0,train
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.900000,0.680000,0.846667,0.423333,0.520000,0.846667,0.760000,-249.7775,0.0,train
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.486667,0.693333,0.526667,0.520000,0.716667,0.706667,0.673333,-247.5775,0.0,train


In [26]:
all_data.tail()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Dem9,Cred1,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,Target,data
56957,18333,Grp102,0.553333,1.043333,1.096667,0.686667,0.673333,0.340000,0.900000,0.643333,...,0.433333,0.660000,0.776667,0.610000,0.690000,0.750000,0.700000,-249.5050,NaN,test
56958,244207,Grp504,1.353333,0.616667,0.276667,0.783333,0.690000,0.650000,0.473333,0.670000,...,0.870000,0.683333,0.690000,0.640000,0.883333,0.663333,0.660000,-248.7525,NaN,test
56959,103277,Grp78,1.083333,0.433333,0.806667,0.490000,0.243333,0.316667,0.533333,0.606667,...,0.063333,0.753333,0.780000,0.603333,0.880000,0.643333,0.676667,-231.0500,NaN,test
56960,273294,Grp134,0.566667,1.153333,0.370000,0.616667,0.793333,0.226667,0.910000,0.696667,...,1.026667,0.626667,0.646667,0.566667,0.616667,0.713333,0.706667,-246.3150,NaN,test
56961,223337,Grp18,1.426667,0.110000,-0.006667,-0.200000,0.983333,1.870000,0.033333,0.963333,...,0.670000,0.770000,0.893333,0.586667,0.616667,0.683333,0.650000,-248.4500,NaN,test


In [27]:
all_data.shape

(284807, 29)

In [28]:
train.isnull().sum()


id                0
Group             0
Per1              0
Per2              0
Per3              0
Per4              0
Per5              0
Per6              0
Per7              0
Per8              0
Per9              0
Dem1              0
Dem2              0
Dem3              0
Dem4              0
Dem5              0
Dem6              0
Dem7              0
Dem8              0
Dem9              0
Cred1             0
Cred2             0
Cred3             0
Cred4             0
Cred5             0
Cred6             0
Normalised_FNT    0
Target            0
data              0
dtype: int64

In [29]:
test.isnull().sum()

id                0
Group             0
Per1              0
Per2              0
Per3              0
Per4              0
Per5              0
Per6              0
Per7              0
Per8              0
Per9              0
Dem1              0
Dem2              0
Dem3              0
Dem4              0
Dem5              0
Dem6              0
Dem7              0
Dem8              0
Dem9              0
Cred1             0
Cred2             0
Cred3             0
Cred4             0
Cred5             0
Cred6             0
Normalised_FNT    0
data              0
dtype: int64

In [30]:
all_data = pd.merge(all_data, geo, on='id', how='left')

In [31]:
all_data = pd.merge(all_data, qsets, on='id', how='left')

In [32]:
all_data = pd.merge(all_data, instance, on='id', how='left')

In [33]:
all_data = pd.merge(all_data, lambdawts, on='Group', how='left')
all_data.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Cred4,Cred5,Cred6,Normalised_FNT,Target,data,geo_score,qsets_normalized_tat,instance_scores,lambda_wt
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.603333,0.686667,0.673333,-245.7500,0.0,train,0.22,-0.70,-0.06,-0.13
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.670000,0.553333,0.653333,-248.0000,0.0,train,-0.25,0.14,0.52,0.66
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.670000,0.686667,0.673333,-233.1250,0.0,train,-0.95,-0.43,1.56,-0.51
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.520000,0.846667,0.760000,-249.7775,0.0,train,0.49,-0.31,0.70,0.72
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.716667,0.706667,0.673333,-247.5775,0.0,train,0.85,-0.63,-0.47,0.60


In [34]:
all_data.isnull().sum()

id                          0
Group                       0
Per1                        0
Per2                        0
Per3                        0
Per4                        0
Per5                        0
Per6                        0
Per7                        0
Per8                        0
Per9                        0
Dem1                        0
Dem2                        0
Dem3                        0
Dem4                        0
Dem5                        0
Dem6                        0
Dem7                        0
Dem8                        0
Dem9                        0
Cred1                       0
Cred2                       0
Cred3                       0
Cred4                       0
Cred5                       0
Cred6                       0
Normalised_FNT              0
Target                  56962
data                        0
geo_score                   0
qsets_normalized_tat        0
instance_scores             0
lambda_wt                   0
dtype: int

In [35]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284807 entries, 0 to 284806
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    284807 non-null  int64  
 1   Group                 284807 non-null  object 
 2   Per1                  284807 non-null  float64
 3   Per2                  284807 non-null  float64
 4   Per3                  284807 non-null  float64
 5   Per4                  284807 non-null  float64
 6   Per5                  284807 non-null  float64
 7   Per6                  284807 non-null  float64
 8   Per7                  284807 non-null  float64
 9   Per8                  284807 non-null  float64
 10  Per9                  284807 non-null  float64
 11  Dem1                  284807 non-null  float64
 12  Dem2                  284807 non-null  float64
 13  Dem3                  284807 non-null  float64
 14  Dem4                  284807 non-null  float64
 15  

In [36]:
train = all_data[all_data['data']=='train']
test = all_data[all_data['data']=='test']

train.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Cred4,Cred5,Cred6,Normalised_FNT,Target,data,geo_score,qsets_normalized_tat,instance_scores,lambda_wt
0,112751,Grp169,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,...,0.603333,0.686667,0.673333,-245.7500,0.0,train,0.22,-0.70,-0.06,-0.13
1,18495,Grp161,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,...,0.670000,0.553333,0.653333,-248.0000,0.0,train,-0.25,0.14,0.52,0.66
2,23915,Grp261,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,...,0.670000,0.686667,0.673333,-233.1250,0.0,train,-0.95,-0.43,1.56,-0.51
3,50806,Grp198,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,...,0.520000,0.846667,0.760000,-249.7775,0.0,train,0.49,-0.31,0.70,0.72
4,184244,Grp228,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,...,0.716667,0.706667,0.673333,-247.5775,0.0,train,0.85,-0.63,-0.47,0.60


In [37]:
test.head()

,id,Group,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,...,Cred4,Cred5,Cred6,Normalised_FNT,Target,data,geo_score,qsets_normalized_tat,instance_scores,lambda_wt
227845,146574,Grp229,-0.300000,1.540000,0.220000,-0.280000,0.570000,0.260000,0.700000,1.076667,...,0.796667,0.823333,0.783333,-249.7500,NaN,test,0.25,-0.43,-0.04,0.76
227846,268759,Grp141,0.633333,0.953333,0.810000,0.466667,0.910000,0.253333,1.040000,0.550000,...,0.630000,0.673333,0.673333,-249.8125,NaN,test,0.43,-0.62,-0.77,0.18
227847,59727,Grp188,1.043333,0.740000,0.860000,1.006667,0.583333,0.616667,0.630000,0.686667,...,0.596667,0.680000,0.670000,-248.1200,NaN,test,1.32,-0.41,0.11,0.39
227848,151544,Grp426,1.283333,0.300000,0.576667,0.636667,0.256667,0.543333,0.356667,0.663333,...,0.320000,0.676667,0.660000,-222.9875,NaN,test,-2.11,0.37,0.33,1.80
227849,155008,Grp443,1.186667,0.326667,0.476667,0.866667,0.436667,0.680000,0.476667,0.686667,...,0.546667,0.650000,0.663333,-196.2200,NaN,test,-2.11,-0.13,-0.37,1.89


In [38]:
x_train = train.drop(['id','Group','Target','data'], axis=1)
y_train = train['Target']

In [39]:
x_test = test.drop(['id','Group','Target','data'], axis=1)
y_test = test['Target']

x_train.head()

,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,Per9,Dem1,...,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,geo_score,qsets_normalized_tat,instance_scores,lambda_wt
0,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,0.863333,0.460000,...,1.010000,0.933333,0.603333,0.686667,0.673333,-245.7500,0.22,-0.70,-0.06,-0.13
1,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,0.190000,0.470000,...,0.690000,0.560000,0.670000,0.553333,0.653333,-248.0000,-0.25,0.14,0.52,0.66
2,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,0.226667,0.660000,...,0.383333,0.763333,0.670000,0.686667,0.673333,-233.1250,-0.95,-0.43,1.56,-0.51
3,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,0.486667,1.096667,...,0.846667,0.423333,0.520000,0.846667,0.760000,-249.7775,0.49,-0.31,0.70,0.72
4,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,0.516667,0.756667,...,0.526667,0.520000,0.716667,0.706667,0.673333,-247.5775,0.85,-0.63,-0.47,0.60


In [40]:
# Data pre-processing 
# missing value Treatment
# feature scaling 
# handeling  the imbalance dataset

In [41]:
y_train.value_counts()

0.0    227451
1.0       394
Name: Target, dtype: int64

In [42]:
x_train.isnull().sum()

Per1                    0
Per2                    0
Per3                    0
Per4                    0
Per5                    0
Per6                    0
Per7                    0
Per8                    0
Per9                    0
Dem1                    0
Dem2                    0
Dem3                    0
Dem4                    0
Dem5                    0
Dem6                    0
Dem7                    0
Dem8                    0
Dem9                    0
Cred1                   0
Cred2                   0
Cred3                   0
Cred4                   0
Cred5                   0
Cred6                   0
Normalised_FNT          0
geo_score               0
qsets_normalized_tat    0
instance_scores         0
lambda_wt               0
dtype: int64

In [43]:
# WE CAN ALSO TRY THIS MODEL BY USING THIS METHODS  

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# I ALSO TRIED THESE METHODS BUT IT IS TAKING NEARLY 5-7 HOURS TO RUN THE MODEL SO BY THIS SITUATION I AM MOVING TO DEEP LEARNING

# IF I DIDNT GET GOOD RESULTS IN DNN THEN AGAIN I WILL COME BACK HEAR 



In [44]:
rf = RandomForestClassifier()
# once you fix this then pls use cross_val_score
rf_score = cross_val_score(rf, x_train, y_train, cv = 10)

In [45]:
# deep neural Network Method - when i have huge dataset i will use this method

import tensorflow as tf
from tensorflow import keras

In [46]:
x_train.head()

,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,Per9,Dem1,...,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,geo_score,qsets_normalized_tat,instance_scores,lambda_wt
0,1.070000,0.580000,0.480000,0.766667,1.233333,1.993333,0.340000,1.010000,0.863333,0.460000,...,1.010000,0.933333,0.603333,0.686667,0.673333,-245.7500,0.22,-0.70,-0.06,-0.13
1,0.473333,1.206667,0.883333,1.430000,0.726667,0.626667,0.810000,0.783333,0.190000,0.470000,...,0.690000,0.560000,0.670000,0.553333,0.653333,-248.0000,-0.25,0.14,0.52,0.66
2,1.130000,0.143333,0.946667,0.123333,0.080000,0.836667,0.056667,0.756667,0.226667,0.660000,...,0.383333,0.763333,0.670000,0.686667,0.673333,-233.1250,-0.95,-0.43,1.56,-0.51
3,0.636667,1.090000,0.750000,0.940000,0.743333,0.346667,0.956667,0.633333,0.486667,1.096667,...,0.846667,0.423333,0.520000,0.846667,0.760000,-249.7775,0.49,-0.31,0.70,0.72
4,0.560000,1.013333,0.593333,0.416667,0.773333,0.460000,0.853333,0.796667,0.516667,0.756667,...,0.526667,0.520000,0.716667,0.706667,0.673333,-247.5775,0.85,-0.63,-0.47,0.60


In [47]:
x_test.head()

,Per1,Per2,Per3,Per4,Per5,Per6,Per7,Per8,Per9,Dem1,...,Cred2,Cred3,Cred4,Cred5,Cred6,Normalised_FNT,geo_score,qsets_normalized_tat,instance_scores,lambda_wt
227845,-0.300000,1.540000,0.220000,-0.280000,0.570000,0.260000,0.700000,1.076667,0.930000,0.156667,...,0.813333,0.776667,0.796667,0.823333,0.783333,-249.7500,0.25,-0.43,-0.04,0.76
227846,0.633333,0.953333,0.810000,0.466667,0.910000,0.253333,1.040000,0.550000,0.543333,0.433333,...,0.703333,0.806667,0.630000,0.673333,0.673333,-249.8125,0.43,-0.62,-0.77,0.18
227847,1.043333,0.740000,0.860000,1.006667,0.583333,0.616667,0.630000,0.686667,0.593333,1.250000,...,0.753333,0.870000,0.596667,0.680000,0.670000,-248.1200,1.32,-0.41,0.11,0.39
227848,1.283333,0.300000,0.576667,0.636667,0.256667,0.543333,0.356667,0.663333,1.156667,1.186667,...,0.606667,0.456667,0.320000,0.676667,0.660000,-222.9875,-2.11,0.37,0.33,1.80
227849,1.186667,0.326667,0.476667,0.866667,0.436667,0.680000,0.476667,0.686667,1.476667,1.213333,...,0.896667,0.566667,0.546667,0.650000,0.663333,-196.2200,-2.11,-0.13,-0.37,1.89


In [48]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc_x_train = sc.fit_transform(x_train)
sc_x_test = sc.fit_transform(x_test)

In [49]:
x_train1 = pd.DataFrame(sc_x_train)
x_test1 = pd.DataFrame(sc_x_test)

In [50]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sc_x_train, y_train, test_size=0.25, random_state=101)

In [51]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(170883, 29)
(56962, 29)
(170883,)
(56962,)


In [52]:
# Building DNN Architecture - APi

# DNN SEQUENCIAL
# SETTING SETUP SEQUENTIAL API FOR BUILDING DEEP NEURAL NETWORK

dnn = tf.keras.models.Sequential()

# connecting to first hidden layer with input layer

dnn.add(tf.keras.layers.Dense(units=6, activation = 'relu'))

# connecting to second hidden layer with first layer

dnn.add(tf.keras.layers.Dense(units= 6 , activation = 'relu'))

# adding output layer 

dnn.add(tf.keras.layers.Dense(units= 1 , activation = 'sigmoid'))

#now connecting with traning dataset by using multilayer perceptron

dnn.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

dnn.fit(x_train , y_train , batch_size= 100 , epochs = 20)

Epoch 1/20
1709/1709 [==============================] - 5s 2ms/step - loss: 0.0721 - accuracy: 0.9851
Epoch 2/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0045 - accuracy: 0.9993
Epoch 3/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0039 - accuracy: 0.9994
Epoch 4/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0036 - accuracy: 0.9993
Epoch 5/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0035 - accuracy: 0.9994
Epoch 6/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0033 - accuracy: 0.9994
Epoch 7/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0032 - accuracy: 0.9994
Epoch 8/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0031 - accuracy: 0.9994
Epoch 9/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.0030 - accuracy: 0.9994
Epoch 10/20
1709/1709 [==============================] - 3s 2ms/step - loss: 0.003

In [53]:
dnn.fit(x_test, y_test, batch_size=100,epochs=10)

Epoch 1/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0029 - accuracy: 0.9994
Epoch 2/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0026 - accuracy: 0.9994
Epoch 3/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0024 - accuracy: 0.9995
Epoch 4/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0022 - accuracy: 0.9995
Epoch 5/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0021 - accuracy: 0.9995
Epoch 6/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0021 - accuracy: 0.9995
Epoch 7/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0020 - accuracy: 0.9995
Epoch 8/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0019 - accuracy: 0.9995
Epoch 9/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0018 - accuracy: 0.9996
Epoch 10/10
570/570 [==============================] - 1s 2ms/step - loss: 0.0017 - accuracy: 0.9995

In [54]:
y_pred = dnn.predict(x_test1)
y_pred = (y_pred>0.5)
y_pred = pd.DataFrame(y_pred)

In [55]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [56]:
accuracy_score(y_test, y_pred)

0.9969102208489871

In [57]:
print(confusion_matrix(y_test, y_pred))

[[56786    87]
 [   89     0]]


In [58]:
y_pred = dnn.predict(x_test)
y_pred = (y_pred>0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [59]:
print(confusion_matrix(y_test, y_pred))

[[56870     3]
 [   16    73]]


In [60]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     56873
         1.0       0.96      0.82      0.88        89

    accuracy                           1.00     56962
   macro avg       0.98      0.91      0.94     56962
weighted avg       1.00      1.00      1.00     56962



In [63]:
# BY THIS DEEP LEARNING METHOD WE ARE GETTING BEST ACCURACY 

# 0.9969102208489871     MEANS  --- 99.691 %

# THIS IS VERY NEAREST TO 100 %

# MEANS I BUILD BEST MODEL FOR THE CLIENT 

# BY THIS MODEL THERE ARE MANY USES 

# 1. CUSTOMERS MONEY WAS SAFE 
# 2. BANK GAINS GOOD MORE WILL BY THIS IS GETS MORE NEW ACCOUNTS AND MONEY DEPOSITS
accuracy_score(y_test, y_pred)


0.9996664442961974

In [67]:
# Fraud on payment card transactions totalled $490.1 million in the 12 months to 30 June 2021 (FY21), an increase of 9.2% from FY20.

# IN A YEAR FROM BANKS $ 490.1 MILLION IS STEELING BY FRAUDLENT TRANSCTIONS 

# AND EVERY YEAR IT IS INCREASING BY 9.2% AS OBSERVED IN 2019,2020,2021 BY AustralianPAYMENTS NETWORK


# BY USING OUR DEEP NURAL NETWORK METHOD 

# ( Total Loss / 100 ) * accuracy of the odel = Amount of Money saved

(490.1 /100)*99.6910 #  =  $ 488.5943032196903 #MILLION 

# ALL BANKS PROFITS PER YEAR INCREASES TO $ 488.5943032196903 MILLION DOLLARS

# DEPENDING ON THE STATISTICS WE CAN REDUCE THE FRAUDLENT TRANSCTIONS VERY HIGH IN A YEAR BY THIS MODEL



488.58559099999997